# Pronosoft Football Data - Extend Database

Extend original Database with additional columns & features:
- Expected Value (EV)
- Match Outcome (1, N or 2)
- Pronosoft Prediction Outcome (0 or 1)

---

---

# Import Lib

In [1]:
import sys
import os
from os import path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

import datetime as dt

---

# Paths

In [2]:
folder_data = '..\Data'
csv_name = folder_data + '\pronosoft_data.csv'
csv_name_extended = folder_data + '\pronosoft_data_extended.csv'

---

# Functions

Find Insert Column Location

In [3]:
def find_insert_loc(col_next_to):
    col_next_loc = list(df_pronosoft_data.columns).index(col_next_to)
    return col_next_loc + 1

Compute Expected Value (EV)

In [4]:
def get_col_ev(col_prob, col_odd):    
    col_ev = col_prob * (col_odd.astype(float) - 1) - (1 - col_prob)
    return np.round(col_ev, 2)

Get Pronosoft Prediction Outcome

In [5]:
def get_col_prediction_team_outcome(col_bet, col_match_outcome):
    col_bet_outcome = []
    
    for idx in range(len(col_bet)):
        if not pd.isna(col_bet[idx]) and col_match_outcome[idx] in col_bet[idx]:
            col_bet_outcome.append(1)
        elif not pd.isna(col_bet[idx]) and col_match_outcome[idx] not in col_bet[idx]:
            col_bet_outcome.append(0)
        else:
            col_bet_outcome.append(None)
    
    return col_bet_outcome

In [6]:
def get_col_prediction_uo_outcome(col_bet, col_score_team_1, col_score_team_2):
    col_bet_outcome = []
    
    for idx in range(len(col_bet)):
        if pd.isna(col_bet[idx]):
            col_bet_outcome.append(None)
        else:
            if (col_score_team_1[idx] + col_score_team_2[idx]) < 2.5 and col_bet[idx] == 'U':
                col_bet_outcome.append(1)
            elif (col_score_team_1[idx] + col_score_team_2[idx]) > 2.5 and col_bet[idx] == 'O':
                col_bet_outcome.append(1)
            else:
                col_bet_outcome.append(0)
    
    return col_bet_outcome

Get Match Outcome

In [7]:
def get_col_match_outcome(col_score_team_1, col_score_team_2):
    col_match_outcome = []
    
    for idx in range(len(col_score_team_1)):
        if col_score_team_1[idx] > col_score_team_2[idx]:
            col_match_outcome.append('1')
        elif col_score_team_1[idx] < col_score_team_2[idx]:
            col_match_outcome.append('2')
        else:
            col_match_outcome.append('N')
            
    return col_match_outcome

Get Probabilities of outcome from Betting Odds

In [8]:
def get_col_prob_from_odds(col_target_outcome, col_other_outcome_1, col_other_outcome_2):    
    
    col_prob_from_odds = 1 / (col_target_outcome * ((1/col_target_outcome) + (1/col_other_outcome_1) + (1/col_other_outcome_2)))
    
    return np.round(col_prob_from_odds, 2)

Modify Field Names

In [9]:
def mod_field_names(new_field_names):
    df = pd.read_csv(csv_name)
    cols = df.columns

    new_names = {cols[i]: new_field_names[i] for i in range(len(new_field_names))}

    df_mod = df_pronosoft_data.rename(columns = new_names)
    df_mod.to_csv(csv_name, index = False)

---

---

# Import CSV to Dataframe

In [10]:
df_pronosoft_data = pd.read_csv(csv_name)
df_pronosoft_data.head()

,date,league,time,team_1_name,team_2_name,team_1_prob,team_1_bet_odds,nul_prob,nul_bet_odds,team_2_prob,team_2_bet_odds,prediction_team_pronosoft,under_prob,under_bet_odds,over_prob,over_bet_odds,prediction_uo_pronosoft,team_1_score,team_2_score
0,01-10-2018,France - Ligue 2,20:45,Troyes,Auxerre,0.39,2.75,0.18,3.10,0.43,2.80,NaN,0.68,1.46,0.32,2.00,U,1,0
1,01-10-2018,Espagne - Liga Espagnole,21:00,Celta Vigo,Getafe,0.33,2.15,0.38,3.20,0.30,3.60,N,0.51,1.53,0.49,1.88,NaN,1,1
2,01-10-2018,Angleterre - Premier League,21:00,Bournemouth,Crystal Palace,0.39,2.20,0.25,3.40,0.36,3.20,NaN,0.40,1.85,0.60,1.65,O,2,1
3,01-10-2018,Italie - Serie A,20:30,Sampdoria,Spal,0.41,1.72,0.25,3.50,0.33,4.80,NaN,0.57,1.67,0.43,1.85,NaN,2,1
4,01-10-2018,Portugal - Primeira Liga,21:15,Aves,Portimonense,0.40,2.60,0.30,3.25,0.30,2.45,NaN,0.69,1.66,0.31,1.73,U,3,0


Modify Field Names

In [11]:
# new_field_names = [ 
#     "date",
#     "league",
#     "time",
#     "team_1_name",
#     "team_2_name",
#     "team_1_prob",
#     "team_1_bet_odds",
#     "nul_prob",
#     "nul_bet_odds",
#     "team_2_prob",
#     "team_2_bet_odds",
#     "prediction_team_pronosoft",
#     "under_prob",
#     "under_bet_odds",
#     "over_prob",
#     "over_bet_odds",
#     "prediction_uo_pronosoft",
#     "team_1_score",
#     "team_2_score"
# ]

# csv_to_mod = csv_name

# mod_field_names(csv_to_mod, new_field_names)

---

# Add Features

Add EV

In [12]:
cols_ev = {
    'team_1_ev': find_insert_loc('team_1_bet_odds'), 
    'nul_ev': find_insert_loc('nul_bet_odds'),
    'team_2_ev': find_insert_loc('team_2_bet_odds'),
    'under_ev': find_insert_loc('under_bet_odds'),
    'over_ev': find_insert_loc('over_bet_odds')
}

counter = 0
for key, value in cols_ev.items():    
    col_ev = get_col_ev(df_pronosoft_data[key[:-2] + 'prob'], df_pronosoft_data[key[:-2] + 'bet_odds'])
    df_pronosoft_data.insert(loc = value + counter, column = key, value = col_ev, allow_duplicates = True)
    
    counter += 1

Add Match Outcome

In [13]:
col_new = 'match_outcome'
col_next_to = 'team_2_score'

col_match_outcome = get_col_match_outcome(df_pronosoft_data['team_1_score'], df_pronosoft_data['team_2_score'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_match_outcome, allow_duplicates = True)

Add Prediction Outcome

In [14]:
col_new = 'prediction_team_pronosoft_outcome'
col_next_to = 'prediction_team_pronosoft'

col_match_outcome = get_col_prediction_team_outcome(df_pronosoft_data['prediction_team_pronosoft'], df_pronosoft_data['match_outcome'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_match_outcome, allow_duplicates = True)

In [15]:
col_new = 'prediction_uo_pronosoft_outcome'
col_next_to = 'prediction_uo_pronosoft'

col_match_outcome = get_col_prediction_uo_outcome(df_pronosoft_data['prediction_uo_pronosoft'], df_pronosoft_data['team_1_score'], df_pronosoft_data['team_2_score'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_match_outcome, allow_duplicates = True)

Get Probabilities of outcome from Betting Odds

**Note**: We can calculate the Probability of Outcome from Betting Odds, which reflects Bookmakers belief & predictions about outcomes. 

Here is an example for Team 1:

\begin{equation*}
Prob\ (Team\_1) =
\frac{1} { Team\_1\_odd \times(\frac{1}{Team\_1\_odd} + \frac{1}{Nul\_odd} + \frac{1}{Team\_2\_odd})}
\end{equation*}

In [16]:
col_new = 'team_1_prob_from_odds'
col_next_to = 'team_1_prob'

col_prob_from_odds = get_col_prob_from_odds(df_pronosoft_data['team_1_bet_odds'], df_pronosoft_data['nul_bet_odds'], df_pronosoft_data['team_2_bet_odds'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_prob_from_odds, allow_duplicates = True)

In [17]:
col_new = 'nul_prob_from_odds'
col_next_to = 'nul_prob'

col_prob_from_odds = get_col_prob_from_odds(df_pronosoft_data['nul_bet_odds'], df_pronosoft_data['team_1_bet_odds'], df_pronosoft_data['team_2_bet_odds'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_prob_from_odds, allow_duplicates = True)

In [18]:
col_new = 'team_2_prob_from_odds'
col_next_to = 'team_2_prob'

col_prob_from_odds = get_col_prob_from_odds(df_pronosoft_data['team_2_bet_odds'], df_pronosoft_data['team_1_bet_odds'], df_pronosoft_data['nul_bet_odds'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_prob_from_odds, allow_duplicates = True)

Get EV from Betting Odds

In [19]:
col_new = 'team_1_ev_from_odds'
col_next_to = 'team_1_ev'

col_ev = get_col_ev(df_pronosoft_data['team_1_prob_from_odds'], df_pronosoft_data['team_1_bet_odds'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_ev, allow_duplicates = True)

In [20]:
col_new = 'nul_ev_from_odds'
col_next_to = 'nul_ev'

col_ev = get_col_ev(df_pronosoft_data['nul_prob_from_odds'], df_pronosoft_data['nul_bet_odds'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_ev, allow_duplicates = True)

In [21]:
col_new = 'team_2_ev_from_odds'
col_next_to = 'team_2_ev'

col_ev = get_col_ev(df_pronosoft_data['team_2_prob_from_odds'], df_pronosoft_data['team_2_bet_odds'])
df_pronosoft_data.insert(loc = find_insert_loc(col_next_to), column = col_new, value = col_ev, allow_duplicates = True)

---

# Save to CSV | Extended Data

In [23]:
df_pronosoft_data.to_csv(csv_name_extended, index = False)

---

---

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('Football_Data_Extended.sqlite')
c = conn.cursor()  # Database saved where '.ipynb' is

# UPDATE DATABASE WITH CSV
df_pronosoft_data.to_sql('Football_Bets', conn, if_exists = 'replace', index=False)

NameError: name 'df_pronosoft_data' is not defined

In [41]:
%load_ext sql
%sql sqlite:///../SQL/Football_Data_Extended.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [53]:
%%sql 

SELECT date, league, team_1_name, team_2_name, team_1_bet_odds, team_1_prob_from_odds, team_1_ev_from_odds, nul_bet_odds, nul_prob_from_odds, nul_ev_from_odds, team_2_bet_odds, team_2_prob_from_odds, team_2_ev_from_odds, team_1_score, team_2_score FROM Football_Bets
ORDER BY team_1_ev_from_odds DESC
LIMIT 100

   sqlite:///../SQL/Football_Data_Extended.sqlite
 * sqlite:///Football_Data_Extended.sqlite
Done.


date,league,team_1_name,team_2_name,team_1_bet_odds,team_1_prob_from_odds,team_1_ev_from_odds,nul_bet_odds,nul_prob_from_odds,nul_ev_from_odds,team_2_bet_odds,team_2_prob_from_odds,team_2_ev_from_odds,team_1_score,team_2_score
16-08-2019,Croatie - Division 1,Inter Zapresic,Dinamo Zagreb,9.5,0.27,1.56,5.4,0.48,1.59,10.3,0.25,1.58,1,2
19-06-2020,Russie - Premier League,Pfc Sochi,FC Rostov,2.6,0.73,0.9,12.0,0.16,0.92,17.0,0.11,0.87,10,1
21-01-2020,Gibraltar - National League - Groupe Relégation,Mons Calpe SC,Manchester 62 FC,3.85,0.41,0.58,5.0,0.31,0.55,5.6,0.28,0.57,3,0
26-04-2019,Malte - Division 1,St. Andrews FC,Gzira United,3.8,0.4,0.52,4.0,0.38,0.52,7.0,0.22,0.54,1,0
18-04-2021,Tunisie - Ligue 1,Cs Sfaxien,Etoile Du Sahel,6.75,0.2,0.35,3.8,0.35,0.33,2.9,0.46,0.33,1,2
07-06-2020,Monténégro - CFL,Fk Petrovac,Iskra Danilovgrad,2.4,0.53,0.27,5.0,0.25,0.25,5.9,0.22,0.3,4,1
18-05-2021,Italie - Serie A,Lazio,Torino,2.3,0.55,0.26,4.33,0.29,0.26,8.0,0.16,0.28,0,0
06-01-2019,France - Coupe de France,Garde Sain,vy Pontivy,60.0,0.02,0.2,26.0,0.04,0.04,1.02,0.95,-0.03,0,4
16-03-2019,Bulgarie - Division 1 - Division 1 Pro,Vereya Stara Zagora,Levski Sofia,60.0,0.02,0.2,13.0,0.07,-0.09,1.03,0.91,-0.06,3,4
18-05-2019,Portugal - Primeira Liga,Feirense,Aves,2.55,0.47,0.2,4.2,0.29,0.22,5.0,0.24,0.2,2,1
